In [ ]:
import trimesh
import numpy as np
import os
import datetime

now = datetime.datetime.now()

In [ ]:
def make_Points(file_path, n_points):
    # STL 파일을 읽어옵니다.
    mesh = trimesh.load(file_path)

    # 표면 위의 점들과 법선벡터를 균일하게 샘플링합니다.
    points, _ = trimesh.sample.sample_surface(mesh, n_points)

    # xyz 좌표의 최소값과 최대값을 구합니다.
    min_xyz = np.min(points, axis=0)
    max_xyz = np.max(points, axis=0)

    # 스케일링 비율을 구합니다.
    scale = 1.0 / np.max(max_xyz - min_xyz)

    # xyz 좌표와 법선벡터를 0~1 범위로 스케일링합니다.
    points_scaled = scale * (points - min_xyz)

    # 스케일링된 좌표와 법선벡터를 출력합니다.
    # print(points_scaled)
    
    return points_scaled

In [ ]:
def point_cloud_to_voxel(point_cloud, n=32):
    # 좌표값 추출
    x = point_cloud[:, 0]
    y = point_cloud[:, 1]
    z = point_cloud[:, 2]

    # 좌표값을 0~n-1 범위의 정수 좌표값으로 변환
    x_idx = np.clip(np.floor(x * n), 0, n-1).astype(int)
    y_idx = np.clip(np.floor(y * n), 0, n-1).astype(int)
    z_idx = np.clip(np.floor(z * n), 0, n-1).astype(int)

    # n*n*n*1 크기의 배열 생성
    voxel = np.zeros((n, n, n, 1), dtype=np.int)

    # 배열의 좌표값에 해당하는 위치에 1을 설정
    voxel[x_idx, y_idx, z_idx, 0] = 1
    
    return voxel

In [ ]:
categories = {
    'airplane': 0,
    'bathtub': 1,
    'bed': 2,
    'bench': 3,
    'bookshelf': 4,
    'bottle': 5,
    'bowl': 6,
    'car': 7,
    'chair': 8,
    'cone': 9,
    'cup': 10,
    'curtain': 11,
    'desk': 12,
    'door': 13,
    'dresser': 14,
    'flower_pot': 15,
    'glass_box': 16,
    'guitar': 17,
    'keyboard': 18,
    'lamp': 19,
    'laptop': 20,
    'mantel': 21,
    'monitor': 22,
    'night_stand': 23,
    'person': 24,
    'piano': 25,
    'plant': 26,
    'radio': 27,
    'range_hood': 28,
    'sink': 29,
    'sofa': 30,
    'stairs': 31,
    'stool': 32,
    'table': 33,
    'tent': 34,
    'toilet': 35,
    'tv_stand': 36,
    'vase': 37,
    'wardrobe': 38,
    'xbox': 39
}

In [ ]:
# Define input and output directories
stl_dir = './ModelNet40stl'
np_dir = './ModelNet40vox32'

# Create np_dir if it does not exist
if not os.path.exists(np_dir):
    os.makedirs(np_dir)

# Loop over categories
for category in categories:
    print('Processing category:', category)

    # Create output directory for current category
    current_np_dir = os.path.join(np_dir, category)
    if not os.path.exists(current_np_dir):
        os.makedirs(current_np_dir)

    # Get file list for current category
    category_dir = os.path.join(stl_dir, category)
    files = os.listdir(category_dir)

    # Loop over files in category
    for file in files:
        # Read stl file
        file_path = os.path.join(category_dir, file)
        now = datetime.datetime.now()
        print(now.strftime('%y%m%d - %H:%M:%S'), ' ', file)
        vertices = make_Points(file_path, 2048)
        pcd = point_cloud_to_voxel(vertices, 32)
        
        # Save npy file
        npy_file_path = os.path.join(current_np_dir, file.split('.')[0] + '.npy')
        np.save(npy_file_path, pcd)